In [2]:
import pandas as pd

# # Função para ler as bases "b" e "c" com delimitadores personalizados
# def read_custom_csv(filename, date_column, separator):
#     return pd.read_csv(filename, parse_dates=[date_column], delimiter=separator)

base_price_coffee = read_custom_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/price_coffee.csv')
# Ler a base "b"
base_ibovespa = read_custom_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibovespa.csv')
# Ler a base "c" sem a especificação do delimitador
base_cidades_all = read_custom_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/cidades_all.csv')


TypeError: read_custom_csv() missing 2 required positional arguments: 'date_column' and 'separator'

In [2]:
# Converter a coluna de datas em datetime
base_a['Data'] = pd.to_datetime(base_a['Data'])
base_b['Data'] = pd.to_datetime(base_b['Data'])
base_c['Data'] = pd.to_datetime(base_c['Data'])

# Criar um conjunto de datas da base "a" para usar como referência
datas_base_a = set(base_a['Data'])

# Filtre as bases "b" e "c" para manter apenas as linhas com datas que existem na base "a"
base_b = base_b[base_b['Data'].isin(datas_base_a)]
base_c = base_c[base_c['Data'].isin(datas_base_a)]


In [3]:
import pandas as pd

# Suponha que a base "c" comece em 2006-08-23 (substitua a data real)
data_inicio_base_b = pd.to_datetime('2006-08-23')

# Converter a coluna de datas em datetime
base_a['Data'] = pd.to_datetime(base_a['Data'])
base_b['Data'] = pd.to_datetime(base_b['Data'])
base_c['Data'] = pd.to_datetime(base_c['Data'])

# Filtre a base "a" para manter apenas as datas a partir da data de início da base "c"
base_a = base_a[base_a['Data'] >= data_inicio_base_b]

# Filtre a base "b" para manter apenas as datas a partir da data de início da base "c"
base_c = base_c[base_c['Data'] >= data_inicio_base_b]


In [4]:
# Exportar base "a" para CSV
base_a.to_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/preco_cafe_preprocessada.csv', index=False)

# Exportar base "b" para CSV
base_b.to_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/clima_patrocinio_preprocessada.csv', index=False)

# Exportar base "c" para CSV
base_c.to_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibovespa_preprocessada.csv', index=False)
